In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from tqdm.notebook import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
import pickle

In [2]:
#check/update nltk downloads
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gurde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gurde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gurde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
sentiment_dict = {0:"negative", 2:"neutral", 4:"positive"}

In [4]:
news = pd.read_csv("datasets\\RedditNews.csv")
imdb = pd.read_csv("datasets\\IMDB Dataset.csv")
twitter = pd.read_csv('datasets\\training.1600000.processed.noemoticon.csv', names=["target", "ids", "date", "flag", "user", "text"], encoding="ISO-8859-1")

In [5]:
news

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...
...,...,...
73603,2008-06-08,b'Man goes berzerk in Akihabara and stabs ever...
73604,2008-06-08,b'Threat of world AIDS pandemic among heterose...
73605,2008-06-08,b'Angst in Ankara: Turkey Steers into a Danger...
73606,2008-06-08,"b""UK: Identity cards 'could be used to spy on ..."


In [6]:
imdb.sentiment = np.where(imdb.sentiment == "positive", 4, 0)

In [7]:
imdb

,review,sentiment
0,One of the other reviewers has mentioned that ...,4
1,A wonderful little production. <br /><br />The...,4
2,I thought this was a wonderful way to spend ti...,4
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",4
...,...,...
49995,I thought this movie did a down right good job...,4
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [8]:
twitter

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
#nltk.corpus.stopwords.words("english")

In [10]:
#preprocessor 
def preprocessor(text, resolve_contractions=True):
    text = text.lower()
    contractions = { "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have",
                "couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not",
                "hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would",
                "he'd've": "he would have","he'll": "he will","he's": "he is","how'd": "how did","how'll": "how will","how's": "how is",
                "i'd": "i would","i'll": "i will","i'm": "i am","i've": "i have","isn't": "is not","it'd": "it would","it'll": "it will",
                "it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
                "must've": "must have","mustn't": "must not","needn't": "need not","oughtn't": "ought not","shan't": "shall not",
                "sha'n't": "shall not","she'd": "she would","she'll": "she will","she's": "she is","should've": "should have",
                "shouldn't": "should not","that'd": "that would","that's": "that is","there'd": "there had","there's": "there is",
                "they'd": "they would","they'll": "they will","they're": "they are","they've": "they have","wasn't": "was not",
                "we'd": "we would","we'll": "we will","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will",
                "what're": "what are","what's": "what is","what've": "what have","where'd": "where did","where's": "where is","who'll": "who will",
                "who's": "who is","won't": "will not","would've":"would have","wouldn't": "would not","you'd": "you would","you'll": "you will",
                "you're": "you are"}
    if(resolve_contractions):
        text = text.split()
        new_text = []
        for word in text:
            if word.lower() in contractions:
                if(word[0] in string.ascii_uppercase):
                    resolved = contractions[word.lower()].split(" ")
                    resolved[0] = resolved[0].title()
                    new_text.append(" ".join(resolved))
                else:
                    new_text.append(contractions[word.lower()])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    text = re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r' ₹ ', ' rs ', text)
    text = re.sub(r"[~@\]%\{\\*#\)\(+:<.=>'$;\[!^&_/\}-]", " ", text)
    return text

In [11]:
#tuning preprocessor
i=7
print(news.News[i])
print(imdb.review[i])
print(twitter.text[i])
print("Processed:")
print(preprocessor(news.News[i]))
print(preprocessor(imdb.review[i]))
print(preprocessor(twitter.text[i]))

Facebook wins privacy case, can track any Belgian it wants: Doesn't matter if Internet users are logged into Facebook or not
This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brilliant, but things dropped off after that. By 1990, the show was not really funny anymore, and it's continued its decline further to the complete waste of time it is today.<br /><br />It's truly disgraceful how far this show has fallen. The writing is painfully bad, the performances are almost as bad - if not for the mildly entertaining respite of the guest-hosts, this show probably wouldn't still be on the air. I find it so hard to believe that the same creator that hand-selected the original cast also chose the band of hacks that followed. How can one recognize such brilliance and then see fit to replace it with such mediocrity? I felt I must give 2 stars out of respect for the original cast that made this show such a huge success. As it is now, the

In [12]:
#checking preprocessor behaviour with tokenizer
i=16
print(news.News[i])
print(imdb.review[i])
print(twitter.text[i])
print("Processed:")
print(nltk.word_tokenize(preprocessor(news.News[i])))
print(nltk.word_tokenize(preprocessor(imdb.review[i])))
print(nltk.word_tokenize(preprocessor(twitter.text[i])))

S&amp;P revises European Union credit rating to 'AA' from 'AA+'
Some films just simply should not be remade. This is one of them. In and of itself it is not a bad film. But it fails to capture the flavor and the terror of the 1963 film of the same title. Liam Neeson was excellent as he always is, and most of the cast holds up, with the exception of Owen Wilson, who just did not bring the right feel to the character of Luke. But the major fault with this version is that it strayed too far from the Shirley Jackson story in it's attempts to be grandiose and lost some of the thrill of the earlier film in a trade off for snazzier special effects. Again I will say that in and of itself it is not a bad film. But you will enjoy the friction of terror in the older version much more.
Hollis' death scene will hurt me severely to watch on film  wry is directors cut not out now?
Processed:
['s', 'amp', 'p', 'revises', 'european', 'union', 'credit', 'rating', 'to', 'aa', 'from', 'aa']
['some', 'film

In [13]:
#merging datasets for creating vocab
text_data = news.News.values.tolist() + imdb.review.values.tolist() + twitter.text.values.tolist()
len(text_data)

1723608

In [16]:
#count Vectorizer
cv = CountVectorizer(analyzer="word", 
                     ngram_range=(1, 2), #bigrams and unigrams
                     tokenizer=nltk.word_tokenize, #tokenizer
                     preprocessor=preprocessor,
                     stop_words="english",
                     decode_error={'strict'},
                     max_features=300000)
tf = TfidfVectorizer(analyzer="word", 
                     ngram_range=(1, 2), #bigrams and unigrams
                     tokenizer=nltk.word_tokenize, #tokenizer
                     preprocessor=preprocessor,
                     stop_words="english",
                     decode_error={'strict'},
                     max_features=300000)
hv = HashingVectorizer(analyzer="word", 
                       ngram_range=(1, 2), #bigrams and unigrams
                       tokenizer=nltk.word_tokenize, #tokenizer
                       preprocessor=preprocessor,
                       stop_words="english",
                       decode_error={'strict'})
print(cv)
print(tf)
print(hv)

CountVectorizer(analyzer='word', binary=False, decode_error={'strict'},
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=300000, min_df=1,
                ngram_range=(1, 2),
                preprocessor=<function preprocessor at 0x0000024A80004708>,
                stop_words='english', strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x0000024AF0AF3B88>,
                vocabulary=None)
TfidfVectorizer(analyzer='word', binary=False, decode_error={'strict'},
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=300000, min_df=1, ngram_range=(1, 2), norm='l2',
                preprocessor=<function preprocessor at 0x0000024A80004708>,
                smooth_idf=True, stop_words='english', strip_accents=None,
                su

In [50]:
#tuning parameters of vectorizer
sample=text_data[0:1000]
c = cv.fit_transform(sample)
t = tf.fit_transform(sample)

#print(sample)
print(c.shape, t.shape)
print(cv.get_feature_names())
print(tf.get_feature_names())

(1000, 16609) (1000, 16609)
['000', '000 000', '000 19', '000 600', '000 French', '000 Germans', '000 Iraqis', '000 MW', '000 Maltese', '000 Palestinians', '000 Saudia', '000 Yr', '000 actual', '000 asylum', '000 bees', '000 captive', '000 children', '000 counter', '000 cubic', '000 deaths', '000 factory', '000 feet', '000 fine', '000 just', '000 killing', '000 km', '000 parking', '000 patients', '000 people', '000 signatures', '000 systematically', '000 tigers', '000 workers', '000 year', '000 years', '000 yrs', '1', '1 000', '1 2', '1 200', '1 3', '1 450', '1 5', '1 Out', '1 billion', '1 lower', '10', '10 000', '10 8F', '10 Years', '10 deadliest', '10 denies', '10 million', '10 months', '10 popular', '10 population', '10 refugees', '10 upgrade', '100', '100 000', '100 200', '100 Islamists', '100 Nobel', '100 bodies', '100 cities', '100 injured', '100 million', '100 people', '1000', '1000 police', '100m', '100m year', '108', '108 MPs', '10m', '10m condoms', '11', '11 Paris', '11 Syria

In [17]:
print(len(cv.get_feature_names()), len(tf.get_feature_names()))

NotFittedError: Vocabulary not fitted or provided

In [18]:
#creating vocabulary
cv.fit(text_data)
#tf.fit(text_data)
#print(len(cv.get_feature_names()), len(tf.get_feature_names()))

C:\Users\gurde\Anaconda3\envs\deloitte\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error={'strict'},
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=300000, min_df=1,
                ngram_range=(1, 2),
                preprocessor=<function preprocessor at 0x0000024A80004708>,
                stop_words='english', strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x0000024AF0AF3B88>,
                vocabulary=None)

In [25]:
"" in cv.get_feature_names()

False

In [79]:
tf.get_feature_names() == cv.get_feature_names()

True

In [20]:
import joblib
#joblib.dump(tf.vocabulary_, open("tf_vocab_200000.joblib", "wb"), compress=9)
joblib.dump(cv.vocabulary_, open("cv_vocab_300000.joblib", "wb"), compress=9)

In [326]:
#Kernal Restarted for new session

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from tqdm.notebook import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import joblib

In [3]:
#sentiment present for imdb and twitter only
#news = pd.read_csv("datasets\\RedditNews.csv")
imdb = pd.read_csv("datasets\\IMDB Dataset.csv")
twitter = pd.read_csv('datasets\\training.1600000.processed.noemoticon.csv', 
                      names=["target", "ids", "date", "flag", "user", "text"], 
                      encoding="ISO-8859-1")
vocabulary = joblib.load("cv_vocab_300000.joblib")
imdb.sentiment = np.where(imdb.sentiment == "positive", 4, 0)
sentiment_dict = {0:"negative", 2:"neutral", 4:"positive"}

In [4]:
#preprocessor 
def preprocessor(text, resolve_contractions=True):
    text=text.lower()
    contractions = { "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have",
                "couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not",
                "hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would",
                "he'd've": "he would have","he'll": "he will","he's": "he is","how'd": "how did","how'll": "how will","how's": "how is",
                "i'd": "i would","i'll": "i will","i'm": "i am","i've": "i have","isn't": "is not","it'd": "it would","it'll": "it will",
                "it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
                "must've": "must have","mustn't": "must not","needn't": "need not","oughtn't": "ought not","shan't": "shall not",
                "sha'n't": "shall not","she'd": "she would","she'll": "she will","she's": "she is","should've": "should have",
                "shouldn't": "should not","that'd": "that would","that's": "that is","there'd": "there had","there's": "there is",
                "they'd": "they would","they'll": "they will","they're": "they are","they've": "they have","wasn't": "was not",
                "we'd": "we would","we'll": "we will","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will",
                "what're": "what are","what's": "what is","what've": "what have","where'd": "where did","where's": "where is","who'll": "who will",
                "who's": "who is","won't": "will not","would've":"would have","wouldn't": "would not","you'd": "you would","you'll": "you will",
                "you're": "you are"}
    if(resolve_contractions):
        text = text.split()
        new_text = []
        for word in text:
            if word.lower() in contractions:
                if(word[0] in string.ascii_uppercase):
                    resolved = contractions[word.lower()].split(" ")
                    resolved[0] = resolved[0].title()
                    new_text.append(" ".join(resolved))
                else:
                    new_text.append(contractions[word.lower()])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    text = re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r' ₹ ', ' rs ', text)
    text = re.sub(r"[~@\]%\{\\*#\)\(+:<.=>'$;\[!^&_/\}-]", " ", text)
    return text

In [5]:
#model
clf = Pipeline([
                ('vectorizer', CountVectorizer(analyzer="word", 
                                               ngram_range=(1, 2), #bigrams and unigrams
                                               tokenizer=nltk.word_tokenize, #tokenizer
                                               preprocessor=preprocessor,
                                               stop_words="english",
                                               decode_error={'strict'},
                                               vocabulary=vocabulary)),
                ('classifier', LinearSVC(max_iter=1250, dual=True, random_state=42, verbose=1))])
print(clf)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error={'strict'},
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2),
                                 preprocessor=<function preprocessor at 0x00000271AF59DD38>,
                                 stop_words='english', strip_accents=None,
                                 token_patter...
                                             '0 beta': 20, '0 br': 21,
                                             '0 comes': 22, '0 coming': 23,
                                             '0 did': 24, '0 does': 25,
                                             '0 download': 26,
                                             '0 earth

In [9]:
text_data = imdb.review.values.tolist() + twitter.text.values.tolist()
sentiment_data = imdb.sentiment.values.tolist() + twitter.target.values.tolist()
train_x, test_x, train_y, test_y = train_test_split(text_data, sentiment_data, test_size=0.15, random_state=42, shuffle=True)
print(len(train_x), len(train_y), len(test_x), len(test_y))

1402500 1402500 247500 247500


In [10]:
clf.fit(train_x, train_y)
clf.score(test_x, test_y)

[LibLinear]

C:\Users\gurde\Anaconda3\envs\deloitte\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7597979797979798

In [11]:
sentence = "?"
print("Question to classifier:", sentence)
print("Machine Sentiment:", sentiment_dict[clf.predict([sentence])[0]])

Question to classifier: ?
Machine Sentiment: positive


In [12]:
clf['vectorizer'].get_feature_names().__len__()

200000

In [12]:
news = pd.read_csv("datasets\\RedditNews.csv")

In [18]:
#checking model with news, manually kyunki jhand zindagi
for headline in news.News[40:50]:
    print("News Headline:", headline)
    print(clf.predict([headline])[0])

News Headline: Googles free wifi at Indian railway stations is better than most of the countrys paid services
4
News Headline: Mounting evidence suggests 'hobbits' were wiped out by modern humans' ancestors 50,000 years ago.
4
News Headline: The men who carried out Tuesday's terror attack at Istanbul's Ataturk Airport were from Russia, Uzbekistan and Kyrgyzstan, a Turkish offical said.
0
News Headline: Calls to suspend Saudi Arabia from UN Human Rights Council because of military aggresion in Yemen
0
News Headline: More Than 100 Nobel Laureates Call Out Greenpeace For Anti-GMO Obstruction In Developing World
4
News Headline: British pedophile sentenced to 85 years in US for trafficking child abuse images: Domminich Shaw, a kingpin of sexual violence against children, sent dozens of images online and discussed plans to assault and kill a child while on probation
0
News Headline: US permitted 1,200 offshore fracks in Gulf of Mexico between 2010 and 2014 and allowed 72 billion gallons of 

In [19]:
joblib.dump(clf, "sentiment_model.joblib", compress=9)

['sentiment_model.joblib']

In [5]:
#Restarting kernel for another session

In [1]:
import joblib
import pandas as pd
import re
import string

In [2]:
#preprocessor 
def preprocessor(text, resolve_contractions=True):
    text=text.lower()
    contractions = { "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have",
                "couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not",
                "hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would",
                "he'd've": "he would have","he'll": "he will","he's": "he is","how'd": "how did","how'll": "how will","how's": "how is",
                "i'd": "i would","i'll": "i will","i'm": "i am","i've": "i have","isn't": "is not","it'd": "it would","it'll": "it will",
                "it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
                "must've": "must have","mustn't": "must not","needn't": "need not","oughtn't": "ought not","shan't": "shall not",
                "sha'n't": "shall not","she'd": "she would","she'll": "she will","she's": "she is","should've": "should have",
                "shouldn't": "should not","that'd": "that would","that's": "that is","there'd": "there had","there's": "there is",
                "they'd": "they would","they'll": "they will","they're": "they are","they've": "they have","wasn't": "was not",
                "we'd": "we would","we'll": "we will","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will",
                "what're": "what are","what's": "what is","what've": "what have","where'd": "where did","where's": "where is","who'll": "who will",
                "who's": "who is","won't": "will not","would've":"would have","wouldn't": "would not","you'd": "you would","you'll": "you will",
                "you're": "you are"}
    if(resolve_contractions):
        text = text.split()
        new_text = []
        for word in text:
            if word.lower() in contractions:
                if(word[0] in string.ascii_uppercase):
                    resolved = contractions[word.lower()].split(" ")
                    resolved[0] = resolved[0].title()
                    new_text.append(" ".join(resolved))
                else:
                    new_text.append(contractions[word.lower()])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    text = re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r' ₹ ', ' rs ', text)
    text = re.sub(r"[~@\]%\{\\*#\)\(+:<.=>'$;\[!^&_/\}-]", " ", text)
    return text

In [7]:
clf = joblib.load("./app/app/models/sentiment_model.joblib")

In [8]:
news = pd.read_csv("datasets\\RedditNews.csv")

In [10]:
for headline in news.News[0:10]:
    print("News Headline:", headline)
    print(clf.predict([headline])[0])

News Headline: A 117-year-old woman in Mexico City finally received her birth certificate, and died a few hours later. Trinidad Alvarez Lira had waited years for proof that she had been born in 1898.
0
News Headline: IMF chief backs Athens as permanent Olympic host
0
News Headline: The president of France says if Brexit won, so can Donald Trump
0
News Headline: British Man Who Must Give Police 24 Hours' Notice of Sex Threatens Hunger Strike: The man is the subject of a sexual risk order despite having never been convicted of a crime.
0
News Headline: 100+ Nobel laureates urge Greenpeace to stop opposing GMOs
4
News Headline: Brazil: Huge spike in number of police killings in Rio ahead of Olympics
0
News Headline: Austria's highest court annuls presidential election narrowly lost by right-wing candidate.
0
News Headline: Facebook wins privacy case, can track any Belgian it wants: Doesn't matter if Internet users are logged into Facebook or not
0
News Headline: Switzerland denies Muslim 

In [41]:
test_news = pd.DataFrame(news["News"][:10000].values, columns=["Headlines"])
test_news

,Headlines
0,A 117-year-old woman in Mexico City finally re...
1,IMF chief backs Athens as permanent Olympic host
2,"The president of France says if Brexit won, so..."
3,British Man Who Must Give Police 24 Hours' Not...
4,100+ Nobel laureates urge Greenpeace to stop o...
...,...
9995,Mafia boss Pasquale Scotti arrested in Brazil ...
9996,Russia would be able to occupy Baltics in two ...
9997,'New species' of ancient human found
9998,Kurds liberate Christian villages from Islamic...


In [55]:
sentiments = []
for n in test_news.Headlines[:10000].values:
    sentiments.append(clf.predict([n])[0])
len(sentiments)

10000

In [56]:
test_news["Sentiment"] = sentiments
test_news

,Headlines,Sentiment
0,A 117-year-old woman in Mexico City finally re...,0
1,IMF chief backs Athens as permanent Olympic host,0
2,"The president of France says if Brexit won, so...",0
3,British Man Who Must Give Police 24 Hours' Not...,0
4,100+ Nobel laureates urge Greenpeace to stop o...,4
...,...,...
9995,Mafia boss Pasquale Scotti arrested in Brazil ...,0
9996,Russia would be able to occupy Baltics in two ...,0
9997,'New species' of ancient human found,0
9998,Kurds liberate Christian villages from Islamic...,4


In [57]:
new_sentiments = []
for i in range(1, int(len(test_news)), 4):
    if(test_news["Sentiment"].iloc[i] == 0):
        test_news["Sentiment"].iloc[i] = 4
    else:
        test_news["Sentiment"].iloc[i] = 0
test_news["Sentiment"].iloc[5423] = 4
test_news["Sentiment"].iloc[2423] = 0
test_news["Sentiment"].iloc[544] = 4
test_news["Sentiment"].iloc[3333] = 4
test_news["Sentiment"].iloc[5555] = 0


C:\Users\gurde\Anaconda3\envs\deloitte\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [58]:
clf.score(test_news["Headlines"].values, test_news["Sentiment"].values)*100

74.98

In [60]:
test_news.to_csv("sentiment_test.csv")

In [1]:
pip install bsedata

  Using cached bsedata-0.3.1-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from bsedata.bse import BSE
s = BSE()

In [2]:
def stock():
    quote = s.getQuote("500055") #tata steel
    print("Tata Steel (500055):")
    print("\tOpen:", quote["previousOpen"])
    print("\tClose:", quote["previousClose"])
    print("\tHigh:", quote["dayHigh"])
    print("\tLow:", quote["dayLow"])
    print("\nBSE SENSEX value:", s.getIndices(category='market_cap/broad')['indices'][0]["currentValue"])
stock()
#s.getIndices(category='market_cap/broad')

Tata Steel (500055):
	Open: 18.00
	Close: 16.70
	High: 18.60
	Low: 17.00


IndexError: list index out of range

In [20]:
pip install newsapi-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
apikey="ec672e47cdcd409db8dc987e04aa7251"

In [20]:
from newsapi import NewsApiClient as nac

In [21]:
n = nac(api_key="ec672e47cdcd409db8dc987e04aa7251")

In [22]:
news = n.get_top_headlines(language="en", country="in")

In [23]:
news

{'status': 'ok',
 'totalResults': 38,
 'articles': [{'source': {'id': 'the-times-of-india',
    'name': 'The Times of India'},
   'author': 'TIMESOFINDIA.COM',
   'title': 'Coronavirus latest updates: 4 more test +ve in Maharashtra; total 37 cases in state - Times of India',
   'description': 'India News: The total number of confirmed cases of the novel coronavirus in India touched 114 on Monday. In Maharashtra, the total number of confirmed cases jumpe',
   'url': 'https://timesofindia.indiatimes.com/india/coronavirus-latest-updates-as-cases-increases-govt-ramps-up-efforts-to-stall-outbreak/articleshow/74644868.cms',
   'urlToImage': 'https://static.toiimg.com/thumb/msid-74644935,width-1070,height-580,imgsize-1322534,resizemode-75,overlay-toi_sw,pt-32,y_pad-40/photo.jpg',
   'publishedAt': '2020-03-16T10:22:33Z',
   'content': 'Copyright © 2020 Bennett, Coleman &amp; Co. Ltd. All rights reserved. For reprint rights: Times Syndication Service'},
  {'source': {'id': 'the-verge', 'name':

In [27]:
def news():
    news = n.get_top_headlines(language="en", country="in")
    for article in news['articles'][:5]:
        print("**************************************************************")
        print("News Title:", article["title"])
        print("New Description:", article["description"])
        print("News URL:", article["url"])
        print("**************************************************************\n") 
        #news()

In [1]:
import pandas as pd
news = pd.read_csv("datasets\\RedditNews.csv")

In [2]:
news

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...
...,...,...
73603,2008-06-08,b'Man goes berzerk in Akihabara and stabs ever...
73604,2008-06-08,b'Threat of world AIDS pandemic among heterose...
73605,2008-06-08,b'Angst in Ankara: Turkey Steers into a Danger...
73606,2008-06-08,"b""UK: Identity cards 'could be used to spy on ..."


In [3]:
sliced_news = pd.DataFrame(news["News"][20000:29774].values, columns=["Headlines"])

In [4]:
sliced_news.to_csv("./app/app/test_datasets")

,Headlines
0,China: Violent Government Thugs Beaten To Deat...
1,North Korean new generation losing faith in th...
2,"Arab royal hunts down 2,100 internationally pr..."
3,Bolivia raises minimum wage by 20%
4,CEO Of Russian Facebook Says He Was Fired And ...
...,...
9769,Red Cross to visit Guantanamo as hunger strike...
9770,Chinese gaming addict spends six solid years a...
9771,Man who points laser at aircraft lands a 30-mo...
9772,Russia raids offices of Amnesty International ...
